## API Tiingo - Diario
###### Helga Zambrana - Abril 2024

#### Importar las librerías necesarias

In [1]:
import requests
from decouple import config
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import datetime

#### Leer el usuario, la contraseña y el token desde la variable de entorno

In [2]:
my_host = config('DATABASE_HOST')
port = config('DATABASE_PORT')
password = config('DATABASE_PASSWORD')
database_name = config('DATABASE_NAME')
user = config('DATABASE_USER')
token = config('TIINGO_TOKEN')

#### Definir la clase TiingoData para manejar los datos de Tiingo

In [6]:
class ListaSimbolos:
    def __init__(self, simbolos):
        self.simbolos = simbolos
    
    def __call__(self):
        return self.simbolos

class TiingoData:
    # Inicializar atributos de la clase
    def __init__(self, token):
        self.token = token

    def get_data(self, lista_simbolos):
        simbolos = lista_simbolos()
        precios = []

        for simbolo in simbolos:
            url = f"https://api.tiingo.com/tiingo/daily/{simbolo}/prices"
            parametros = {
                'startDate': '2023-01-01',
                'endDate': datetime.date.today().isoformat(),
                'format': 'json',
            }
            headers = {
                'Authorization': f'Token {self.token}'  # Agregar el token enmascarado en los headers
            }

            try:
                respuesta = requests.get(url, params=parametros, headers=headers)
                respuesta.raise_for_status()  # Lanzar una excepción si la solicitud no es exitosa
                datos = respuesta.json()
                
                # Verificar la estructura de los datos
                if isinstance(datos, list) and all(isinstance(item, dict) for item in datos):
                    for dato in datos:
                        dato['ticker'] = simbolo  # Agregar el símbolo a cada fila porque si no, no lo trae
                    precios.extend(datos)
                else:
                    print(f"Los datos recibidos para el símbolo {simbolo} no están en el formato esperado.")
            except requests.exceptions.RequestException as e:
                print(f"Error al hacer la solicitud para el símbolo {simbolo}:", e)

        if precios:
            # Convertir los datos en un DataFrame de pandas
            df = pd.DataFrame(precios)
            # Filtrar las columnas no deseadas
            columnas_no_deseadas = ['adjClose', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor']
            df = df.drop(columns=columnas_no_deseadas, errors='ignore')
            # Filtrar y renombrar las columnas necesarias
            columnas = {
                'date': 'fecha',
                'ticker': 'simbolo',
                'open': 'apertura',
                'high': 'maximo',
                'low': 'minimo',
                'close': 'cierre',
                'volume': 'volumen',
            }
            df.rename(columns=columnas, inplace=True)
            # Convertir la columna de fecha a tipo datetime
            df['Fecha'] = pd.to_datetime(df['Fecha'])
            # Llamar a la instancia de ListaSimbolos para imprimir los símbolos
            print(lista_simbolos())
            
            return df
        else:
            return None

#### Conectarse con Amazon Redshift y crear tabla (si no existe)

In [7]:
class BaseDatosRedshift:
    def __init__(self, host, port, database_name, user, password):
        self.host = host
        self.port = port
        self.database_name = database_name
        self.user = user
        self.password = password
        self.engine = None

    def conectar(self):
        try:
            self.engine = create_engine(f'postgresql+psycopg2://{self.user}:{self.password}@{self.host}:{self.port}/{self.database_name}')
            print("Conexión a la base de datos exitosa.")
        except Exception as e:
            print(f"Error al crear la conexión a la base de datos:", e)

    def crear_tabla_tiingo(self):
        if self.conn is not None:
            try:
                cur = self.conn.cursor()
                cur.execute("""
                CREATE TABLE IF NOT EXISTS hvzambrana_coderhouse.tiingo (
                    simbolo VARCHAR(10) DISTKEY NOT NULL,
                    fecha_ingesta TIMESTAMP DEFAULT CURRENT_TIMESTAMP, 
                    fecha TIMESTAMP SORTKEY NOT NULL,
                    apertura FLOAT(15) NULL,
                    maximo FLOAT(15) NULL,
                    minimo FLOAT(15) NULL,
                    cierre FLOAT(15) NULL,
                    volumen INTEGER NULL
                );
                """)
                self.conn.commit()
                print("Tabla creada exitosamente.")
            except Exception as e:
                print(f"Error al crear la tabla:", e)
        else:
            print("No hay conexión a la base de datos.")

    def cargar_datos(self, df):
        if self.engine is not None:
            try:
                df.to_sql('tiingo', self.engine, schema='hvzambrana_coderhouse', if_exists='replace', index=False)
                print("Datos cargados exitosamente.")
            except Exception as e:
                print(f"Error al cargar los datos en la tabla:", e)
        else:
            print("No hay conexión a la base de datos.")

    def cerrar_conexion(self):
        if self.conn is not None:
            self.conn.close()
            print("Conexión cerrada correctamente.")
        else:
            print("No hay conexión para cerrar.")

#### Obtener datos de Tiingo y cargarlos en Redshift

In [8]:
# Lista de símbolos a consultar
simbolos = ['AMZN', 'AAPL', 'DIS', 'GOOGL', 'JNJ', 'MCD', 'MELI', 'MSFT', 'NVDA', 'TSLA']
lista_simbolos = ListaSimbolos(simbolos)

# Crear instancia de TiingoData
tiingo_data = TiingoData(token)

# Obtener y mostrar los datos de Tiingo
datos_tiingo_df = tiingo_data.get_data(lista_simbolos)

# Crear instancia de la clase BaseDatosRedshift con los datos de conexión
bd_redshift = BaseDatosRedshift(my_host, port, database_name, user, password)

# Conectar con la base de datos
bd_redshift.conectar()

# Crear tabla en la base de datos
bd_redshift.crear_tabla_tiingo()

# Cargar los datos en Redshift
bd_redshift.cargar_datos(datos_tiingo_df)

# Cerrar conexión
bd_redshift.cerrar_conexion()

KeyError: 'Fecha'